In [19]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [20]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [21]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(29, 'documents')
(9, 'classes', [u'acp_report', u'goodbye', u'greeting', u'medication_dosage', u'medication_dosage_search', u'options', u'pharmacy_search', u'system_maintenance', u'thanks'])
(57, 'unique stemmed words', [u"'s", u'a', u'acp', u'admin', u'adv', u'anyon', u'ar', u'by', u'bye', u'can', u'car', u'could', u'dat', u'day', u'display', u'do', u'dos', u'find', u'for', u'get', u'go', u'good', u'goodby', u'hello', u'help', u'hi', u'how', u'i', u'is', u'lat', u'list', u'loc', u'maint', u'man', u'me', u'med', u'mod', u'nam', u'op', u'paramet', u'pharm', u'plan', u'provid', u'report', u'run', u'search', u'see', u'setup', u'show', u'system', u'thank', u'that', u'ther', u'to', u'want', u'what', u'you'])


In [22]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_chatbot_redsamurai_medical_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('chatbot_redsamurai_medical_model.tflearn')

Training Step: 3999  | total loss: 0.01184 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.01184 - acc: 1.0000 -- iter: 24/29
Training Step: 4000  | total loss: 0.01178 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.01178 - acc: 1.0000 -- iter: 29/29
--
INFO:tensorflow:/notebooks/chatbot_redsamurai_medical_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [7]:
p = bow("I'm looking for pharmacy", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[u'acp_report', u'goodbye', u'greeting', u'medication_dosage', u'medication_dosage_search', u'options', u'pharmacy_search', u'system_maintenance', u'thanks']


In [8]:
print(model.predict([p]))

[[  6.64305139e-11   9.61143343e-09   1.43495843e-01   1.08581188e-08
    2.99136038e-04   1.74134993e-03   4.79923964e-01   3.74539465e-01
    2.20836256e-07]]


In [9]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "chatbot_redsamurai_medical_training_data", "wb" ) )